In [1]:
from conditional_feinsum import init_spn
import torch
from torch.autograd import Variable
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import ImageNet
from torchvision.transforms import Compose, CenterCrop, Resize, ToTensor
from utils import set_einet_weights, extract_image_patches, get_surrounding_patches
import numpy as np
from nns import MLP
import config
import torch.nn.functional as F
from PIL import Image

In [2]:
device = torch.device('cuda:0')
einet = init_spn(device)

{0}


In [3]:
num_params = 0
for n, p in einet.named_parameters():
    print(n)
    print(p.shape)
    num_params += np.prod(list(p.shape))
num_params

einet_layers.0.ef_array.params
torch.Size([64, 3, 1, 6])
einet_layers.1.params
torch.Size([3, 3, 3, 4])
einet_layers.2.params
torch.Size([3, 3, 1, 2])
einet_layers.3.params
torch.Size([1, 1, 2])


1280

In [4]:
transform = Compose([ToTensor(), Resize(112), CenterCrop(112)])
imagenet = ImageNet('/storage-01/datasets/imagenet/', transform=transform)
clusters = np.load('/storage-01/ml-jseng/imagenet-clusters/vit_cluster_minibatch_10K.npy')
img_ids = np.argwhere(clusters == 2400).flatten()
subset = Subset(imagenet, img_ids)
loader = DataLoader(subset, 32)

In [5]:
def train(einet, train_loader, num_epochs, i, j, device):

    """
        Train ConFeinsum
    """
    out_dims = []
    for i, p in enumerate(einet.parameters()):
        if i == 0:
            nv, k, a, th = p.shape
            # split mean and variance parameters as variance
            # gets relu applied (cannot be negative)
            out_dims.append([nv, k, a, th // 2])
            out_dims.append([nv, k, a, th // 2])
        else:
            out_dims.append(list(p.shape))

    in_dim = (3*config.num_dims*config.num_vars) + 1000
    mlp = MLP(in_dim, out_dims, [256, 512]).to(device)

    optim = torch.optim.Adam(mlp.parameters(), 0.01)

    for epoch in range(num_epochs):
        total_ll = 0.0
        for x, y in train_loader:
            optim.zero_grad()
            x = x.to(device)
            y = y.to(device)
            # patch images and obtain 8x8 patches
            patches = extract_image_patches(x, config.height, config.width)
            x_in = patches[:, :, i, j]
            x_prev = get_surrounding_patches(patches, i, j, device)
            # flatten all previous patches and concat along feature dimension for NN
            x_prev = [x.squeeze().reshape(x_in.shape[0], -1) for x in x_prev]
            x_prev = torch.cat(x_prev, dim=1)
            y_oh = F.one_hot(y, num_classes=1000)
            x_prev = torch.cat([x_prev, y_oh], dim=1)
            einet_params = mlp(x_prev)
            
            ll = 0.
            x_in = x_in.permute((0, 2, 3, 1))
            x_in = x_in.reshape(x_in.shape[0], config.num_vars, config.num_dims)


            # set einet parameters for each batch as parameters differ
            # for each sample
            for k in range(x.shape[0]):
                einet_param = [ep[k] for ep in einet_params]
                einet = set_einet_weights(einet, einet_param)

                # evaluate log-likelihood
                x_in_img = x_in[k].unsqueeze(0)

                ll += einet(x_in_img).sum()

            ll.backward()
            total_ll += ll
            #print(einet.einet_layers[0].ef_array.ll.grad.shape)
            #print([(n, p.grad) for n, p in einet.named_parameters()])
            #print([e.grad for e in einet_params])
            print([(n, p.grad) for n, p in mlp.named_parameters()])
            optim.step()
        print(f"Epoch {epoch} \t LL: {total_ll}")

    return einet, mlp

In [6]:
device = torch.device('cuda:0')
einet = init_spn(device)
einet, mlp = train(einet, loader, 5, 2, 2, device)

{0}


/home/ml-jseng/anaconda3/envs/federated-spn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[('linear_layers.0.weight', None), ('linear_layers.0.bias', None), ('linear_layers.1.weight', None), ('linear_layers.1.bias', None), ('heads.0.weight', None), ('heads.0.bias', None), ('heads.1.weight', None), ('heads.1.bias', None), ('heads.2.weight', None), ('heads.2.bias', None), ('heads.3.weight', None), ('heads.3.bias', None), ('heads.4.weight', None), ('heads.4.bias', None)]
[('linear_layers.0.weight', None), ('linear_layers.0.bias', None), ('linear_layers.1.weight', None), ('linear_layers.1.bias', None), ('heads.0.weight', None), ('heads.0.bias', None), ('heads.1.weight', None), ('heads.1.bias', None), ('heads.2.weight', None), ('heads.2.bias', None), ('heads.3.weight', None), ('heads.3.bias', None), ('heads.4.weight', None), ('heads.4.bias', None)]
[('linear_layers.0.weight', None), ('linear_layers.0.bias', None), ('linear_layers.1.weight', None), ('linear_layers.1.bias', None), ('heads.0.weight', None), ('heads.0.bias', None), ('heads.1.weight', None), ('heads.1.bias', None), (

KeyboardInterrupt: 

In [ ]:
for i in range(10):
    x, y = subset[i]
    x = x.unsqueeze(0)
    patches = extract_image_patches(x, 8, 8)
    x = patches[0, :, 2, 2]
    s = get_surrounding_patches(patches, 2, 2)[0][0]
    x = x * 255
    x = x.permute(1, 2, 0)
    s = s * 255
    s = s.permute(1, 2, 0)
    img = x.numpy().astype(np.uint8)
    pimg = Image.fromarray(img)
    pimg.show()
    imgs = s.numpy().astype(np.uint8)
    pimgs = Image.fromarray(imgs)
    pimgs.show()

In [12]:
a = torch.randn(3, 12, 3)
b = torch.cat((a, a**2), dim=-1)

In [14]:
b[0, 0]

tensor([-0.5499,  0.3395, -0.2652,  0.3024,  0.1153,  0.0703])

In [19]:
a = [1, 2, 3]
a[2:]

[3]

In [6]:
from torch.autograd import Variable
import torch.nn

In [14]:
w = Variable(torch.randn(3, 3), requires_grad=True)
x = torch.randn(5, 3)
y = torch.mm(x, w)
x_ = torch.randn(3, 5)
y_ = torch.mm(x_, y)
l = torch.sum(y_)
l.backward()

In [15]:
w.grad

tensor([[-0.1417, -0.1417, -0.1417],
        [ 0.6123,  0.6123,  0.6123],
        [ 0.3511,  0.3511,  0.3511]])

In [1]:
from ceinsum import EinsumNetwork, Graph, EinetMixture
import torch
from torch.autograd import Variable
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import ImageNet
from torchvision.transforms import Compose, CenterCrop, Resize, ToTensor
import numpy as np
from nns import MLP
import config
import torch.nn.functional as F
from PIL import Image

def init_spn(device):
    """
        Build a SPN (implemented as an einsum network). The structure is either
        the same as proposed in https://arxiv.org/pdf/1202.3732.pdf (referred to as
        poon-domingos) or a binary tree.

        In case of poon-domingos the image is split into smaller hypercubes (i.e. a set of
        neighbored pixels) where each pixel is a random variable. These hypercubes are split further
        until we operate on pixel-level. The spplitting is done randomly. For more information
        refer to the link above.
    """

    if config.structure == 'poon-domingos':
        pd_delta = [[config.height / d, config.width / d] for d in config.pd_num_pieces]
        graph = Graph.poon_domingos_structure(shape=(config.height, config.width), delta=pd_delta)
    elif config.structure == 'binary-trees':
        graph = Graph.random_binary_trees(num_var=config.num_vars, depth=config.depth, num_repetitions=config.num_repetitions)
    elif config.structure == 'flat-binary-tree':
        graph = Graph.binary_tree_spn(shape=(config.height, config.width))
    else:
        raise AssertionError("Unknown Structure")

    args = EinsumNetwork.Args(
            num_var=config.num_vars,
            num_dims=config.num_dims,
            num_classes=1,
            num_sums=config.K,
            num_input_distributions=config.K,
            exponential_family=config.exponential_family,
            exponential_family_args=config.exponential_family_args,
            online_em_frequency=config.online_em_frequency,
            online_em_stepsize=config.online_em_stepsize)
    
    in_dim = (3*config.num_dims*config.num_vars) + 1000
    out_dims = [[config.num_vars, config.num_dims, 1, 3],
                [config.num_vars, config.num_dims, 1, 3], 
                [config.num_dims, config.num_dims, config.num_dims, 4],
                [config.num_dims, config.num_dims, 1, 2],
                [1, 1, 2]]
    mlp = MLP(in_dim, out_dims, [256, 512]).to(device)
    einet = EinsumNetwork.EinsumNetwork(graph, mlp, args)
    einet.initialize()
    einet.to(device)
    return einet

In [2]:
transform = Compose([ToTensor(), Resize(112), CenterCrop(112)])
imagenet = ImageNet('/storage-01/datasets/imagenet/', transform=transform)
clusters = np.load('/storage-01/ml-jseng/imagenet-clusters/vit_cluster_minibatch_10K.npy')
img_ids = np.argwhere(clusters == 2400).flatten()
subset = Subset(imagenet, img_ids)
loader = DataLoader(subset, 32)

In [9]:
def train(einet, train_loader, num_epochs, i, j, device):

    """
        Train ConFeinsum
    """

    optim = torch.optim.Adam(einet.param_nn.parameters(), 0.0001)

    for epoch in range(num_epochs):
        total_ll = 0.0
        for x, y in train_loader:
            optim.zero_grad()
            x = x.to(device)
            y = y.to(device)
            
            ll = einet(x, y, i, j)
            ll.backward()

            torch.nn.utils.clip_grad.clip_grad_norm_(einet.param_nn.parameters(), 1.)

            total_ll += ll
            #print(einet.einet_layers[0].ef_array.ll.grad.shape)
            #print([(n, p.grad) for n, p in einet.named_parameters()])
            #print([e.grad for e in einet_params])
            #if torch.isnan(ll):
            #    print(x)
            #    print([(n, p) for n, p in einet.named_parameters()])
            optim.step()
        print(f"Epoch {epoch} \t LL: {total_ll}")

    return einet

In [10]:
device = torch.device('cuda:0')
einet = init_spn(device)
einet = train(einet, loader, 5, 2, 2, device)

{0}


/home/ml-jseng/anaconda3/envs/federated-spn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 0 	 LL: 889.8013916015625
Epoch 1 	 LL: 875.5518798828125
Epoch 2 	 LL: 853.3012084960938
Epoch 3 	 LL: 816.2379150390625
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False)
tensor([[[[ 1.9543,  1.1128,  1.6635, -1.9972, -1.4968, -2.1063]],

         [[ 2.0179,  0.6993,  0.2842, -2.2048, -1.4901, -1.2029]],

         [[ 0.1338,  0.6649,  1.1244, -1.2689, -1.4098, -1.4710]]],


        [[[ 2.9661,  1.6989,  0.4491, -2.8677, -2.0426, -1.3558]],

         [[ 4.9348,  0.9646,  0.4408, -4.7240, -1.3430, -1.0660]],

         [[ 0.4585,  0.1105,  1.0299, -1.1222, -1.1081, -1.6710]]],


        [[[ 0.5940,  3.4700,  4.0368, -1.1694, -3.3590, -3.9132]],

         [[ 0.3891,  0.5651,  1.8896, -1.2498, -1.3941, -1.8666]],

         [[ 0.8943,  0.3092,  0.3325, -1.3343, -1.0259, -1.2611]]],


        ...,


        [[[ 1.0290,  0.1934,  0.4802, -1.5813, -1.0238, -1.0967]],

         [[ 0.6587,  1.4598,  1.5264, -1.2259, -1.6965, -1.8763]],

         [[ 1.8154,  1.9085,

In [7]:
torch.log(torch.tensor(-2*-2e2))

tensor(5.9915)